**Run this code on google colab and and first of all mount on specific folder**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/track')
print(os.getcwd())

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import pandas as pd
import random
import ntpath

In [ ]:
datadir='/content/drive/MyDrive/Colab Notebooks/track'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()

In [ ]:
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head()

In [ ]:
num_bins = 25
hist, bins = np.histogram(data['steering'], num_bins)
# make center at 0
matrix_add =(bins[:-1] + bins[1:])
center = matrix_add * 0.5
#print("center: ", center)
plt.bar(center, hist, width=0.05)

#to control bias value in steering for 0, set threshold
samples_per_bin = 200
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))
plt.show()

In [ ]:
print('total data: ', len(data))
remove_list_ = []
for j in range(num_bins):
  list_ = []
  for i in range(len(data['steering'])):
    if (data['steering'][i] >= bins[j]) and (data['steering'][i] <= bins[j+1]):
      list_.append(i)
  list_ = shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list_.extend(list_)
print('removed : ', len(remove_list_))
data.drop(data.index[remove_list_], inplace=True)
print('remaining data: ', len(data))

hist, bins = np.histogram(data['steering'], num_bins)
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))
plt.show()

In [ ]:
#print(data.iloc[0])
def load_image_steering(datadir, data):
  image_path = []
  steering=[]

  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0],indexed_data[1],indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip())) 
    steering.append(float(indexed_data[3]))

  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

image_paths, steerings = load_image_steering(datadir + '/IMG', data)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=6)
print('Training samples: {} \n Valid samples: {}'.format(len(X_train), len(X_valid)))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(y_train, bins=num_bins, width=0.05, color='blue')
axes[0].set_title('Training set')
axes[1].hist(y_valid, bins=num_bins, width=0.05, color='red')
axes[0].set_title('Validation set')

In [ ]:
def img_preprocess(img):
  img = mpimg.imread(img)
  # delete irrelivant features from image
  img = img[60:135, :,: ] 
  # change color space to YUV because of using NVIDIA model architecture
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  # apply gaussian blur for smoothing and reducing noise
  img = cv2.GaussianBlur(img, (3, 3), 0)
  # for faster computation reduce the image size
  img = cv2.resize(img, (200, 66)) # nvidia input image size
  # finally apply normalization
  img = img/255 
  return img

In [ ]:
# test by a signle image
image = image_paths[50]
original_image = mpimg.imread(image)
preprocessed_image = img_preprocess(image)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_image)
axis[0].set_title('Original Image')
axis[1].imshow(preprocessed_image)
axis[1].set_title('Preprocessed Image')


In [ ]:
# now mapping images
X_train = np.array(list(map(img_preprocess, X_train)))
X_valid = np.array(list(map(img_preprocess, X_valid)))

In [ ]:
plt.imshow(X_train[random.randint(0, len(X_train))])
plt.axis('off')
print(X_train.shape)

In [ ]:
def nvidia_model():
  model = Sequential()
  model.add(Convolution2D(24, (5, 5), strides=(2,2), input_shape=(66, 200, 3), activation='elu'))
  model.add(Convolution2D(36, (5, 5), strides=(2,2), activation='elu'))
  model.add(Convolution2D(48, (5, 5), strides=(2,2), activation='elu'))
  model.add(Convolution2D(64, (3, 3), activation='elu'))

  model.add(Convolution2D(64, (3, 3), activation='elu'))
  model.add(Dropout(0.5))

  model.add(Flatten())

  model.add(Dense(100, activation='elu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(50, activation='elu'))  
  model.add(Dropout(0.5))

  model.add(Dense(10, activation='elu'))  
  model.add(Dropout(0.5))
  
  model.add(Dense(1))

  optimizer = Adam(learning_rate=0.001)
  model.compile(loss='mse', optimizer=optimizer)
  return model


In [ ]:
model = nvidia_model()
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), batch_size=100, verbose=1, shuffle=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')

In [ ]:
model.save('model.h5')

In [ ]:
from google.colab import files
files.download('model.h5')